In [5]:
import csv
import re

In [72]:
with open('yourFile.csv', "r+", encoding="utf-8") as file:
    readCSV = csv.reader(file)
    bankName = ""; bic = ""; bankCode = ""; rowBefore = "" # redundant
    equalBanks = []
    counter = 0
    
    for row in readCSV:
        print("first row: ", row[1]) 
        print("first bank name: ", bankName)
        counter = counter + 1
        print(counter)
        
        # same bank name
        if bankName == row[1]:
            if not equalBanks:
                equalBanks += [rowBefore]
            print("banks are equal")
            equalBanks += [row]

        # different bank name
        else:
            # looks in array of same banks for redundancy
            # unbedingt liste leeren am ende von redundancy loeschung
            if re.search(bankName, str(equalBanks)):
                print("test for further process")
                # check for same bankcode and emtpy BIC (has to be at least one bic!)
                for line in equalBanks: 
                    for column in equalBanks:
                        print()                     # TODO
                    print("equalBanks: ", line)

            # different bank names
            else:
                print("bank names different, go on")
        
        print("array: ", equalBanks)
        bankName = row[1]
        bic = row[2]
        bankCode = row[5]
        rowBefore = row
        print()